In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams, colors
from tqdm import tqdm
import emcee
import corner
import plotly.express as px
import plotly.graph_objects as go
import re
import glob

%matplotlib inline
# figure size in inches
rcParams['figure.figsize'] = 12,8
rcParams['axes.formatter.useoffset'] = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Discussion of tidal decay theory

A while back we had some problems modelling the decay of Kelt 1. We got some results that said it should be decaying, but there's a plethora of evidence to suggest that brown giant Kelt-1b is tidally synchronized with its host star.

The general equation for tidal decay that is quoted in almost every tidal decay related paper out there does not take into the relative rotational speeds of star and planet. This is my attempt to explain why they don't take it into account, why we should, and how we can improve this.

#### Simple derivation of tidal decay equation:

1. A torque is induced on the planet due to the fact that the gravitational potential is NOT axisymmetric due to tidal bulges in the star.

\begin{aligned}
T & =M_p a \times -\frac{\partial \phi}{\partial \theta} \\
& =\frac{3}{2} \frac{G M_p}{a}\left(\frac{R}{a}\right)^5 k \sin 2\varepsilon \\
& =\frac{3}{2} \frac{G M_p}{a}\left(\frac{R}{a}\right)^5 \frac{k}{Q}
\end{aligned}

$k$ is a love number the models the amount of surface deformation of the star. This is related to its rigidity and elastic modulus.

$\epsilon$ is the tidal lag angle which describes the phase difference between the star's rotation and its tidal bulge. This is determined by how the star responds to the gravitational driving force of the planet.

We usually assume the tidal lag angle is a constant such that $sin(2\epsilon)$ can be subsumed into a constant known as the tidal dissipation factor Q. It describes the ratio of energy dissipated to the peak energy stored in a single cycle of a forced harmonic osciallator.

More often in exoplanet literature, $k/Q$ is combined into a single constant $Q'$ since we cannot separate out the physics due to rigidity with the physics due to tidal oscillation response for exoplanets.

2. The work done by the torque decreases the orbital energy of the planet:

\begin{aligned}\dot{E}=T(\omega-n)\end{aligned}

3. Total energy of the system (which is not conserved due to heating/friction as the tidal bulge moves across the star):

\begin{aligned}
& E=\frac{1}{2} I \omega^2+\frac{1}{2} \frac{G M_* M}{a} \\
& \dot{E}=I \omega \dot{\omega}+\frac{1}{2} M_n^2 a \dot{a}
\end{aligned}

4. Total angular momentum of the system (which is conserved):

\begin{aligned}
& L=I \omega+M n a^2 \\
& \dot{L}=I \dot{\omega}+\frac{1}{2} \operatorname{Mna} \dot{a}=0
\end{aligned}

Combining these four equations yields the often quoted tidal decay equation:

$$
\dot{a}=\operatorname{sign}(\omega-n) \frac{3 k}{Q} \frac{M}{M_*}\left(\frac{R}{a}\right)^5 n a
$$

This is the equation I have been basing my theoretical predictions on. However there's clearly a big flaw here. The Q term encompasses a lot of physics of how the star responds to a gravitational force, and is by **no means solely a property of the internal structure of the star.**

The details of how Q is modelled is the interesting part. Typically the star's tidal response is modelled as a forced harmonic oscillator:

$$
\frac{d^2 x}{d t^2}=-\omega_0^2 x-\frac{1}{\tau} \frac{d x}{d t}+\frac{F_0}{m} \cos \omega t
$$

Where $\omega$ is the frequency of the driving force i.e. the planetary orbital frequency, and $\omega_{0}$ is the natural frequency of the star i.e. its rotational frequency.

Which gives a solution of the form:

$$
x=A \cos (\omega t+\varepsilon)
$$

$$
\sin \varepsilon=-\frac{\omega}{\tau}\left[\left(\omega_0^2-\omega^2\right)^2+\left(\frac{\omega}{\tau}\right)^2\right]^{-\frac{1}{2}}
$$

If $\left(\frac{\omega}{\tau}\right)^2 \ll \omega^2, \omega_0^2 $:

$$ \sin \varepsilon \propto \frac{1}{\omega_0^2-\omega^2}$$

If $\omega^2 \ll \omega_0^2$:
$$\sin \varepsilon=\text { const. }$$


This means that to assume Q is constant is to assume that the natural frequency of the star (it's rotational speed) is much less than the orbital speed which is not true for systems like Kelt-1.

#### What can we do about this?

* Typically Q' is quoted around 10^5 or 10^6. Perhaps we could adjust these values to create a new "angular velocity adjusted Q" which is roughly constant. We could then re-run our theory with this new "independent" constant factor and take into account rotational velocities. Unfortunately rotational velocities of stars tend to be quite hard to measure and so very uncertain.
* We could make use of this when trying to infer results from "observed" values of Q.

Actually I'm not sure the above is correct - natural frequency of the star does not refer just to its rotational frequency, but also the internal properties that cause its tidal bulge to move non-instantaneously from one angle to the next.